In [10]:
import pandas as pd
import numpy as np

# 读取 npz 文件
data = np.load('./data/SWAT/train_swat.npz')
# ['train_x', 'train_target', 'val_x', 'val_target', 'mean', 'std']
data.files
# 获取 train_x 数组并查看其形状
# train_x = data['train_x']
# npy数据格式是一个四维的数组[N，H，W, C]，其中N代表数据集的总数，H, W，C分别代表每一张图片对应的长、宽、以及通道数。
# # Original train_x shape: (4195, 51, 1, 30)
# # print("Original train_x shape:", train_x.shape)


['train_x', 'train_target', 'val_x', 'val_target', 'mean', 'std']

In [12]:
train_x = data['train_x']
train_x.shape

(4195, 51, 1, 30)

In [14]:
num_samples, dim1, dim2, dim3 = train_x.shape
train_x[0, :, 0, 0]

array([9.94713047e-01, 9.92589827e-02, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 5.55775802e-01, 1.50029207e-01, 7.62462073e-01,
       9.88857665e-01, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       9.88261034e-01, 9.74696236e-01, 7.88396575e-01, 1.89189189e-01,
       1.00000000e+00, 1.17647059e-01, 1.17647059e-01, 0.00000000e+00,
       1.00000000e+00, 7.77777778e-01, 1.00000000e+00, 1.45189680e-01,
       8.79706643e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.41267589e-02, 9.99595257e-01,
       4.84499999e-01, 7.67483907e-02, 1.41870649e-01, 2.07765336e-01,
       9.50586419e-01, 8.62548172e-01, 0.00000000e+00, 0.00000000e+00,
       9.93586027e-01, 7.15514865e-01, 1.00000000e+00, 1.33112351e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [15]:
train_x[0, 0, :, 0]

array([0.99471305])

In [16]:
train_x[0, 0, 0, :]

array([0.99471305, 0.98979154, 0.98649391, 0.99241428, 0.9875675 ,
       0.98697567, 0.99434756, 0.98690712, 0.9893513 , 0.9922855 ,
       0.98598096, 0.98521682, 0.99384086, 0.9900781 , 0.98496348,
       0.99057025, 0.19898705, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])